The goal of this notebook is to fit the continuous variables for all combinations of the categorical variables (part A) and then draw
from the distributions new continuous variables values (part B). These new values will be used later to create a new population in another notebook. Note: there is a section flagged as IGNORE because that section is still under development and specific to the NHANES population...just ignore that section...

In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import multivariate_normal

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM"
dataDir = microsimDir+"/NOTEBOOKS/DATA"

#contains person-year information from a Microsim NHANES simulation
df = pd.read_csv(dataDir+"/nhanes-as-gaussians-01.csv")

In [7]:
#df.columns.to_list()
df.shape

(5448, 22)

In [3]:
categoricalVars = ["gender", "smokingStatus", "raceEthnicity", "statin",'education',
                  'alcoholPerWeek','anyPhysicalActivity','antiHypertensiveCount']
continuousVars = ['age', 'hdl', 'bmi', 'totChol', 'trig', 'a1c', 'ldl', 'waist', 'creatinine', 'sbp', 'dbp']

In [4]:
#for col in categoricalVars:
#    print(df[col].value_counts())

## PART A: FIT DATA TO GAUSSIANS

### STEP 1

In [5]:
%%time
#iterate over all combinations of the categorical variables
#not all combinations of categorical variables include 1 or more nhanes persons...
#so find the combinations that actually include 1 or more persons
nCategories = 0
sizesList = list()
for gender in set(df["gender"].tolist()):
    for smoking in set(df["smokingStatus"].tolist()):
        #just to have an idea how fast this is going....
        print(gender,smoking)
        for raceEthnicity in set(df["raceEthnicity"].tolist()):
            for statin in set(df["statin"].tolist()):
                for education in set(df["education"].tolist()):
                    for alcoholPerWeek in set(df["alcoholPerWeek"].tolist()):
                        for anyPhysicalActivity in set(df["anyPhysicalActivity"].tolist()):
                            for antiHypertensiveCount in set(df["antiHypertensiveCount"].tolist()):
                                
                                nCategories += 1
                                
                                dfForGroup = df.loc[(df["gender"]==gender) & 
                                                        (df["smokingStatus"]==smoking) &
                                                        (df["raceEthnicity"]==raceEthnicity) &
                                                        (df["statin"]==statin) &
                                                        (df["education"]==education) &
                                                        (df["alcoholPerWeek"]==alcoholPerWeek) &
                                                        (df["anyPhysicalActivity"]==anyPhysicalActivity) &
                                                        (df["antiHypertensiveCount"]==antiHypertensiveCount), :].copy()
                                
                                #the number of rows will show me how many persons I have in this group of categorical variables
                                size = dfForGroup.shape[0]
                                
                                if size>0:
                                    group = [gender, smoking, raceEthnicity, statin, education, alcoholPerWeek, 
                                         anyPhysicalActivity, antiHypertensiveCount]
                                    names = dfForGroup["name"].tolist()
                                    sizesList += [[group, size, names]]
                                                    

1 0
1 1
1 2
2 0
2 1
2 2
CPU times: user 5.91 s, sys: 5.75 ms, total: 5.92 s
Wall time: 5.93 s


In [8]:
#rename the variable....
groupsList = sizesList
#if things went ok, this should be the same as the size of the original dataset
totalPeople = sum(list(map( lambda x: x[1], sizesList)))
totalPeople==df.shape[0]

True

In [ ]:
#ignore this part
#trying this out...for now....but this would apply only on nhanes not on other cases...

#with (open(dataDir+"/nhanesMeans-4.csv", "w") as meansFile, open(dataDir+"/nhanesCovs-4.csv","w") as covsFile,
#      open(dataDir+"/nhanesMin-4.csv", "w") as minFile, open(dataDir+"/nhanesMax-4.csv", "w") as maxFile):
#    pd.DataFrame(categoricalVars+["weight"]+continuousVars).T.to_csv(meansFile, header=False, index=False)
#    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(covsFile, header=False, index=False)
#    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(minFile, header=False, index=False)
#    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(maxFile, header=False, index=False)
#    for i,groupWithSize in enumerate([groupsList[0]]):
#        if i%500==0:
#            print(i)
#        group = groupWithSize[0]
#        size = groupWithSize[1]
    
#        gender = group[0]
#        smoking = group[1]
#        raceEthnicity = group[2]
#        statin = group[3]
#        education = group[4]
#        alcoholPerWeek = group[5]
#        anyPhysicalActivity = group[6] 
#        antiHypertensiveCount = group[7]
    
#        dataDf = df.loc[(df["gender"]==gender) & 
#                                (df["smokingStatus"]==smoking) &
#                                (df["raceEthnicity"]==raceEthnicity) &
#                                (df["statin"]==statin) &
#                                (df["education"]==education) &
#                                (df["alcoholPerWeek"]==alcoholPerWeek) &
#                                (df["anyPhysicalActivity"]==anyPhysicalActivity) &
#                                (df["antiHypertensiveCount"]==antiHypertensiveCount), 
#                                                        continuousVars+["name"]].copy()
#        data = np.array(dataDf)       

In [ ]:
#dataDf.head()

In [ ]:
#nhanesDf = pd.read_stata(microsimDir + "/CODE/microsim/microsim/data/fullyImputedDataset.dta")
#nhanesDf = nhanesDf.rename(columns={"level_0":"name"})
#dataDf = dataDf.merge(nhanesDf[["name","WTINT2YR"]], on="name", how="inner").copy()

In [ ]:
#dataDf["weight"] = dataDf["WTINT2YR"].astype('int')
#dataDf.head()

In [ ]:
#for row in range(dataDf.shape[0]):
    
#    copied_df = pd.DataFrame(dataDf.iloc[row].repeat(2)).reset_index(drop=True)

    # Concatenate the original DataFrame with the copied rows
#    dataDf = pd.concat([dataDf, copied_df], ignore_index=True)

In [ ]:
#copied_df.T

In [ ]:
#dataDf.head()

In [ ]:
#dataDf.applymap(lambda x: np.random.default_rng().uniform(0.8*x, 1.2*x))

### STEPS 2 AND 3

In [9]:
#fit the continuous variables for all combinations of categorical variables
#some fits will result in singular covariance matrices, that is ok for now
#I am writing the fit parameters to files because I want to see what process I need to use
#if others give us distributions of their populations and we need to create Person objects from those distributions
with (open(dataDir+"/nhanesMeans.csv", "w") as meansFile, open(dataDir+"/nhanesCovs.csv","w") as covsFile,
      open(dataDir+"/nhanesMin.csv", "w") as minFile, open(dataDir+"/nhanesMax.csv", "w") as maxFile):
    
    #write the column names on the files, min, max, mean, covariance
    pd.DataFrame(categoricalVars+["weight"]+continuousVars).T.to_csv(meansFile, header=False, index=False)
    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(covsFile, header=False, index=False)
    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(minFile, header=False, index=False)
    pd.DataFrame(categoricalVars+continuousVars).T.to_csv(maxFile, header=False, index=False)
    
    #I am now iterariting over the combination of categorical variables that include more than 1 person
    #no need to iterate over all possible combinations...
    for i,groupWithSize in enumerate(groupsList):
        if i%500==0:
            print(i)
        group = groupWithSize[0]
        size = groupWithSize[1]
    
        gender = group[0]
        smoking = group[1]
        raceEthnicity = group[2]
        statin = group[3]
        education = group[4]
        alcoholPerWeek = group[5]
        anyPhysicalActivity = group[6] 
        antiHypertensiveCount = group[7]
        
        #STEP 2
        
        #get the rows from the dataframe for specifically this combination of categorical variables
        data = np.array(df.loc[(df["gender"]==gender) & 
                                (df["smokingStatus"]==smoking) &
                                (df["raceEthnicity"]==raceEthnicity) &
                                (df["statin"]==statin) &
                                (df["education"]==education) &
                                (df["alcoholPerWeek"]==alcoholPerWeek) &
                                (df["anyPhysicalActivity"]==anyPhysicalActivity) &
                                (df["antiHypertensiveCount"]==antiHypertensiveCount), 
                                                        continuousVars].copy())
     
        #do the fit
        distMean, distCov = multivariate_normal.fit(data)
        
        #STEP 3
        
        #write the min for this group
        distMinDf = pd.DataFrame( np.min(data,axis=0) )
        pd.concat(
                [ pd.DataFrame([gender, smoking, raceEthnicity, statin, 
                                education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount]).T,
                      distMinDf.T],axis=1, ignore_index=True).to_csv(minFile, header=False, index=False)
        minFile.write("\n\n")
        
        #write the max for this group
        distMaxDf = pd.DataFrame( np.max(data,axis=0) )
        pd.concat(
                [ pd.DataFrame([gender, smoking, raceEthnicity, statin, 
                                education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount]).T,
                      distMaxDf.T],axis=1, ignore_index=True).to_csv(maxFile, header=False, index=False)
        maxFile.write("\n\n")
        
        #write the mean for this group
        #1 row per block because means are of shape (1, # continuous variables)
        distMeanDf = pd.DataFrame(distMean)
        pd.concat(
                    [ pd.DataFrame([gender, smoking, raceEthnicity, statin, 
                                    education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount, size/totalPeople]).T,
                      distMeanDf.T],axis=1, ignore_index=True).to_csv(meansFile, header=False, index=False)
        meansFile.write("\n\n")
        
        #write the covariance matrix for this group
        #several rows per block because covs are of shape (# continuous variable, # continuous variable)
        distCovDf = pd.DataFrame(distCov)
        pd.concat(
                [ pd.concat([pd.DataFrame([gender, smoking, raceEthnicity, statin,
                                           education, alcoholPerWeek, anyPhysicalActivity, antiHypertensiveCount]).T]*len(continuousVars),ignore_index=True),
                  distCovDf.T],axis=1, ignore_index=True).to_csv(covsFile, header=False, index=False)
        covsFile.write("\n\n")

0
500
1000
1500


## PART B: USE GAUSSIANS TO CREATE A NEW POPULATION REPRESENTATION 

In [10]:
meansDf = pd.read_csv(dataDir+"/nhanesMeans.csv", header=0)
covsDf = pd.read_csv(dataDir+"/nhanesCovs.csv", header=0)
minDf = pd.read_csv(dataDir+"/nhanesMin.csv", header=0)
maxDf = pd.read_csv(dataDir+"/nhanesMax.csv", header=0)

In [11]:
meansDf[continuousVars].describe()

,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
count,1611.000000,1611.000000,1611.000000,1611.000000,1611.000000,1611.000000,1611.000000,1611.000000,1611.000000,1611.000000,1611.000000
mean,55.985339,50.595477,28.897159,203.552362,160.532213,5.821042,123.069421,98.957847,0.875937,132.117713,72.145019
std,17.100626,13.462886,5.577041,36.584872,110.356597,1.195587,31.883402,13.076731,0.772782,20.498374,10.290928
min,18.000000,19.000000,16.090000,76.000000,29.000000,3.900000,30.000000,60.800000,0.200000,85.333333,41.333333
25%,42.881944,41.000000,25.492500,181.000000,102.000000,5.200000,103.376471,90.672222,0.600000,117.704545,66.160494
50%,57.600000,49.000000,28.011053,202.000000,137.277778,5.500000,121.000000,97.833333,0.771667,128.666667,71.407407
75%,70.000000,57.333333,31.320000,222.000000,186.750000,5.900000,140.000000,106.233333,0.900000,143.166667,77.916667
max,85.000000,138.000000,67.830000,480.000000,1538.000000,15.100000,354.000000,162.200000,10.100000,220.000000,120.666667


In [ ]:
#covsDf.head()
#minDf.head()
#meansDf.head()

In [12]:
#flag the distributions that have a singular covariance matrix
for groupWithSize in groupsList:
    group = groupWithSize[0]
    size = groupWithSize[1]
    
    gender = group[0]
    smoking = group[1]
    raceEthnicity = group[2]
    statin = group[3]
    education = group[4]
    alcoholPerWeek = group[5]
    anyPhysicalActivity = group[6] 
    antiHypertensiveCount = group[7]
    
    gCov = covsDf.loc[ (covsDf["gender"]==gender) &
             (covsDf["smokingStatus"]==smoking) &
             (covsDf["raceEthnicity"]==raceEthnicity) &
             (covsDf["statin"]==statin) &
             (covsDf["education"]==education) &
             (covsDf["alcoholPerWeek"]==alcoholPerWeek) &
             (covsDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (covsDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    #if np.linalg.matrix_rank(gCov) < gCov.shape[0]:
    #    groupWithSize += [True]
    #else:
    #    groupWithSize += [False]
        
    #if (np.linalg.matrix_rank(gCov) < gCov.shape[0]) | (not np.all(np.linalg.eig(gCov)[0]>0)) | (not np.allclose(gCov, gCov.T)):
    if not np.all(np.linalg.eig(gCov)[0]>10**(-3)):
        groupWithSize += [True]
    else:
        groupWithSize += [False]

In [13]:
sum(list(map(lambda x: x[3], groupsList)))

1531

In [14]:
#for the groups that have a singular covariance matrix find a group with a non-singular covariance matrix
#that can be used as a replacement
for groupWithSize in groupsList:
    group = groupWithSize[0]
    size = groupWithSize[1]
    singular = groupWithSize[3]
    
    if singular:
        #print(group)
        gender = group[0]
        smoking = group[1]
        raceEthnicity = group[2]
        statin = group[3]
        education = group[4]
        alcoholPerWeek = group[5]
        anyPhysicalActivity = group[6] 
        antiHypertensiveCount = group[7]
        
        gMean = meansDf.loc[ (meansDf["gender"]==gender) &
             (meansDf["smokingStatus"]==smoking) &
             (meansDf["raceEthnicity"]==raceEthnicity) &
             (meansDf["statin"]==statin) &
             (meansDf["education"]==education) &
             (meansDf["alcoholPerWeek"]==alcoholPerWeek) &
             (meansDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (meansDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
        
        #find the alternative group that maximizes the probability of the mean, 
        #the alternative group will be the one that will later be used to draw from
        altGroupsList = list()
        for altGroupWithSize in groupsList:
            altGroup = altGroupWithSize[0]
            altSize = altGroupWithSize[1]
            altSingular = altGroupWithSize[3]
            
            if not altSingular:
                altGender = altGroup[0]
                altSmoking = altGroup[1]
                altRaceEthnicity = altGroup[2]
                altStatin = altGroup[3]
                altEducation = altGroup[4]
                altAlcoholPerWeek = altGroup[5]
                altAnyPhysicalActivity = altGroup[6] 
                altAntiHypertensiveCount = altGroup[7]
    
                altGMean = meansDf.loc[ (meansDf["gender"]==altGender) &
                                     (meansDf["smokingStatus"]==altSmoking) &
                                     (meansDf["raceEthnicity"]==altRaceEthnicity) &
                                     (meansDf["statin"]==altStatin) &
                                     (meansDf["education"]==altEducation) &
                                     (meansDf["alcoholPerWeek"]==altAlcoholPerWeek) &
                                     (meansDf["anyPhysicalActivity"]==altAnyPhysicalActivity) &
                                     (meansDf["antiHypertensiveCount"]==altAntiHypertensiveCount), continuousVars]
    
                altGCov = covsDf.loc[ (covsDf["gender"]==altGender) &
                                   (covsDf["smokingStatus"]==altSmoking) &
                                   (covsDf["raceEthnicity"]==altRaceEthnicity) &
                                   (covsDf["statin"]==altStatin) &
                                   (covsDf["education"]==altEducation) &
                                   (covsDf["alcoholPerWeek"]==altAlcoholPerWeek) &
                                   (covsDf["anyPhysicalActivity"]==altAnyPhysicalActivity) &
                                   (covsDf["antiHypertensiveCount"]==altAntiHypertensiveCount), continuousVars]
    
                gDist = multivariate_normal(np.array(altGMean)[0], np.array(altGCov), allow_singular=False)
        
                altGroupsList += [[altGroup, gDist.pdf(gMean)]] #[altGroup, gDist.pdf(group)]
    
        probabilities = list(map(lambda x: x[1], altGroupsList))
        maxProb = max(probabilities)
        altGroupWithMaxProb = altGroupsList[probabilities.index(maxProb)][0]
        groupWithSize += [altGroupWithMaxProb]
    else:
        groupWithSize += [None]

In [15]:
#draw from the distributions to get all continuous variables for all persons
#if a draw happens to be outside reasonable bounds, then re-draw until the draw is within reasonable bounds

personsDf = pd.DataFrame(data=None, columns= ["name"]+categoricalVars+continuousVars)

for iii, groupWithSize in enumerate(groupsList):
    if iii%500==0:
        print(iii)
    group = groupWithSize[0]
    size = groupWithSize[1]
    names = groupWithSize[2]
    singular = groupWithSize[3]
    altGroup = groupWithSize[4]
    
    if singular:
        gender = altGroup[0]
        smoking = altGroup[1]
        raceEthnicity = altGroup[2]
        statin = altGroup[3]
        education = altGroup[4]
        alcoholPerWeek = altGroup[5]
        anyPhysicalActivity = altGroup[6] 
        antiHypertensiveCount = altGroup[7]
    else:
        gender = group[0]
        smoking = group[1]
        raceEthnicity = group[2]
        statin = group[3]
        education = group[4]
        alcoholPerWeek = group[5]
        anyPhysicalActivity = group[6] 
        antiHypertensiveCount = group[7]
    
    gMean = meansDf.loc[ (meansDf["gender"]==gender) &
             (meansDf["smokingStatus"]==smoking) &
             (meansDf["raceEthnicity"]==raceEthnicity) &
             (meansDf["statin"]==statin) &
             (meansDf["education"]==education) &
             (meansDf["alcoholPerWeek"]==alcoholPerWeek) &
             (meansDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (meansDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    gCov = covsDf.loc[ (covsDf["gender"]==gender) &
             (covsDf["smokingStatus"]==smoking) &
             (covsDf["raceEthnicity"]==raceEthnicity) &
             (covsDf["statin"]==statin) &
             (covsDf["education"]==education) &
             (covsDf["alcoholPerWeek"]==alcoholPerWeek) &
             (covsDf["anyPhysicalActivity"]==anyPhysicalActivity) &
             (covsDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    gDist = multivariate_normal(np.array(gMean)[0], np.array(gCov), allow_singular=True)
    
    if size>4:
        gMin = minDf.loc[ (minDf["gender"]==group[0]) &
                 (minDf["smokingStatus"]==group[1]) &
                 (minDf["raceEthnicity"]==group[2]) &
                 (minDf["statin"]==group[3]) &
                 (minDf["education"]==group[4]) &
                 (minDf["alcoholPerWeek"]==group[5]) &
                 (minDf["anyPhysicalActivity"]==group[6]) &
                 (minDf["antiHypertensiveCount"]==group[7]), continuousVars]
    
        gMax = maxDf.loc[ (maxDf["gender"]==group[0]) &
                 (maxDf["smokingStatus"]==group[1]) &
                 (maxDf["raceEthnicity"]==group[2]) &
                 (maxDf["statin"]==group[3]) &
                 (maxDf["education"]==group[4]) &
                 (maxDf["alcoholPerWeek"]==group[5]) &
                 (maxDf["anyPhysicalActivity"]==group[6]) &
                 (maxDf["antiHypertensiveCount"]==group[7]), continuousVars]
    else:
        gMin = minDf.loc[ (minDf["gender"]==gender) &
                 (minDf["smokingStatus"]==smoking) &
                 (minDf["raceEthnicity"]==raceEthnicity) &
                 (minDf["statin"]==statin) &
                 (minDf["education"]==education) &
                 (minDf["alcoholPerWeek"]==alcoholPerWeek) &
                 (minDf["anyPhysicalActivity"]==anyPhysicalActivity) &
                 (minDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
        gMax = maxDf.loc[ (maxDf["gender"]==gender) &
                 (maxDf["smokingStatus"]==smoking) &
                 (maxDf["raceEthnicity"]==raceEthnicity) &
                 (maxDf["statin"]==statin) &
                 (maxDf["education"]==education) &
                 (maxDf["alcoholPerWeek"]==alcoholPerWeek) &
                 (maxDf["anyPhysicalActivity"]==anyPhysicalActivity) &
                 (maxDf["antiHypertensiveCount"]==antiHypertensiveCount), continuousVars]
    
    #nDraws = int(popSize*size/totalPointsInGroups)
    nDraws = size
    nDrawsNeeded = nDraws
    draws=None
    while nDrawsNeeded!=0:
        if draws is None:
            #draw from the Gaussians
            draws = gDist.rvs(size=nDrawsNeeded)
            #or draw from a uniform-tried it and does not model the NHANES population well...
            #draws = np.random.default_rng().uniform(gMin["age"],gMax["age"],nDrawsNeeded)[np.newaxis, :]
            #for var in continuousVars[1:]:
            #    draws = np.concatenate([draws ,
            #                            np.random.default_rng().uniform(gMin[var],gMax[var],nDrawsNeeded)[np.newaxis, :] ])
            #draws = draws.T
            
        else:
            #print(f"{i} redrawing")
            if len(draws.shape)==1:
                draws = draws.reshape((1,len(continuousVars)))
            if (nDrawsNeeded==1):
                draws = np.concatenate( (draws, gDist.rvs(size=nDrawsNeeded).reshape((1,len(continuousVars)))), axis=0 )
            else:
                draws = np.concatenate( (draws, gDist.rvs(size=nDrawsNeeded)), axis=0 )
           
        if nDraws==1:
            draws = draws.reshape((1,len(continuousVars)))
            
        rowsOutOfBounds = np.array([False]*nDraws)
        for i, bound in enumerate(np.array(gMin)[0]):
            rowsOutOfBounds = rowsOutOfBounds | (draws[:,i]<0.9*bound)
        for i, bound in enumerate(np.array(gMax)[0]):
            rowsOutOfBounds = rowsOutOfBounds | (draws[:,i]>1.1*bound)
        nDrawsNeeded = nDraws - np.sum(~rowsOutOfBounds)
        draws = draws[~rowsOutOfBounds,:]    
    
    if nDraws == 0:
        pass
    elif nDraws==1:
        #print(nDraws)
        #personRowCont = pd.DataFrame(draws).T
        personRowCont = pd.DataFrame(draws)
    else:
        #print(nDraws)
        personRowCont = pd.DataFrame(draws)
    if nDraws>0:
        personRowCont.columns=continuousVars
        personRowCat = pd.concat([pd.DataFrame(group).T]*nDraws, ignore_index=True)
        personRowCat.columns = categoricalVars
        personRow = pd.concat( [pd.Series(names), personRowCat, personRowCont], axis=1).rename(columns={0:"name"})
        personsDf = pd.concat([personsDf,personRow])
    

0
500
1000
1500


In [16]:
personsDf["age"] = round(personsDf["age"]).astype('int')

In [17]:
personsDf.head()

,name,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,41909,1,0,1,False,1,0,0,0.0,54,41.167309,25.694482,214.167769,210.549305,5.389349,148.330353,102.545480,0.688871,131.688014,78.255367
1,42688,1,0,1,False,1,0,0,0.0,51,46.384726,31.935565,164.732679,110.189826,5.286215,81.605992,112.511680,0.762213,137.004879,74.640230
2,43025,1,0,1,False,1,0,0,0.0,36,47.408049,28.315046,260.486802,126.640687,5.668748,177.731299,99.632114,0.857567,121.974670,90.400776
3,43390,1,0,1,False,1,0,0,0.0,40,45.379713,30.468159,215.360517,117.954243,5.686785,135.276734,103.727511,0.715434,113.273055,70.062711
4,44501,1,0,1,False,1,0,0,0.0,74,53.576906,22.736810,223.833927,63.617831,6.194046,170.082194,95.710354,0.696987,123.342909,81.288198


In [18]:
personsDf.to_csv(dataDir+"/nhanes-as-gaussians-02.csv", index=False)
#personsDf.to_csv(dataDir+"/nhanes-persons-from-uniforms.csv", index=False)